In [1]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM  
tokenizer = AutoTokenizer.from_pretrained("llmware/bling-1b-0.1")  
model = AutoModelForCausalLM.from_pretrained("llmware/bling-1b-0.1")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
directory_path = r'/home/sdm/Desktop/shakib/KUET LLM/data/'
file_pattern = '*.docx'  
file_paths = glob.glob(directory_path + file_pattern)
file_paths

['/home/sdm/Desktop/shakib/KUET LLM/data/rag4_dhiman.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag2V3_2k20.docx',
 '/home/sdm/Desktop/shakib/KUET LLM/data/rag1_sadia_omar.docx']

In [4]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [5]:
all_doc=[]
for x in file_paths:
    loader = Docx2txtLoader(x)
    documents=loader.load()
    # print(((documents)[0]).page_content)
    all_doc.append(str(documents[0].page_content))
docs='\n\n'.join(all_doc)

In [6]:
RAG.index(
    collection=[docs],
    index_name="bling_colbert",
    max_document_length=256,
    split_documents=True,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Mar 23, 11:15:36] #> Creating directory .ragatouille/colbert/indexes/bling_colbert 


[Mar 23, 11:15:38] [0] 		 #> Encoding 190 passages..
[Mar 23, 11:15:39] [0] 		 avg_doclen_est = 177.55262756347656 	 len(local_sample) = 190
[Mar 23, 11:15:39] [0] 		 Creating 2,048 partitions.
[Mar 23, 11:15:39] [0] 		 *Estimated* 33,734 embeddings.
[Mar 23, 11:15:39] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/bling_colbert/plan.json ..
used 20 iterations (0.1652s) to cluster 32049 items into 2048 clusters
[Mar 23, 11:15:39] Loading

0it [00:00, ?it/s]

[Mar 23, 11:15:41] [0] 		 #> Encoding 190 passages..


1it [00:00,  3.19it/s]
100%|██████████| 1/1 [00:00<00:00, 3518.71it/s]

[Mar 23, 11:15:41] #> Optimizing IVF to store map from centroids to list of pids..
[Mar 23, 11:15:41] #> Building the emb2pid mapping..
[Mar 23, 11:15:41] len(emb2pid) = 33735



100%|██████████| 2048/2048 [00:00<00:00, 150610.77it/s]

[Mar 23, 11:15:41] #> Saved optimized IVF to .ragatouille/colbert/indexes/bling_colbert/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/bling_colbert'

In [7]:
retriever = RAG.as_langchain_retriever(k=3)

In [9]:
# from langchain_core.prompts import ChatPromptTemplate
# template='''
# <human>: " {context} "\n" {question} "\n" + "<bot>:
# '''
# prompt = ChatPromptTemplate.from_template(template)

In [29]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [30]:
from langchain import HuggingFacePipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 100,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
                )
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.3})

In [31]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# torch.cuda.empty_cache()

In [32]:
import pandas as pd
import tqdm
dat=[]
cnt=0
df=pd.read_excel(r"data/KUET information2k20.xlsx")
df.head()
for id,ques in tqdm.tqdm(zip(df['id'],df['Question'])):
    ans=rag_chain.invoke(ques)
    dat.append({
        "id":id,
        "question":ques,
        "answer":ans
    })
    if cnt==20:
        break
    cnt+=1

20it [02:57,  8.86s/it]


In [33]:
sa=pd.DataFrame(dat)
model_ans="bling_rag_without_finetuneV2"
sa.to_excel(f"data//{model_ans}.xlsx",index=False)